In [ ]:
from pymongo import MongoClient
import pandas as pd  
import numpy as np 
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

client = MongoClient('localhost', 27017)
db = client['twitter']

pipeline=[{ "$project": {  "id_user" : 1, "originalTweet" : 1, "hashtag" : 1, "mention" : 1} }]

cursor_list = list(db['tweets'].aggregate(pipeline))

In [ ]:
tweets = pd.DataFrame(cursor_list)
data = pd.DataFrame(columns=['Tweets'])
data['Tweets'] = tweets['originalTweet']

In [ ]:
from utils import tokenize_and_stem
from nltk.corpus import stopwords

myStopWords=set(stopwords.words('english')+list(['http','https']))

docs = data['Tweets']

documents = [tokenize_and_stem(s, stopwords=myStopWords) for s in docs]

In [ ]:
#### TODO --->  Prova con Stemming e senza

#### tagME ---> NER

### reti DOc2Vec  Kaggle

Topic Modeling is primarily concerned with identifying ‘topics’ (in this sense, a pattern of co-occurring words; ultimately, a topic is a distribution over a given vocabulary) in a corpus (= set of documents).

For the LDA model, we need a document-term matrix (a gensim dictionary) and all articles in vectorized format (we will be using a bag-of-words approach - constructing a vector representation of an article 'doc2bow(text)' )

In [ ]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(documents)
#dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
print(dictionary)

A corpus of all Tweet is constructed and vectorized

In [ ]:
corpus = [dictionary.doc2bow(text) for text in documents]
# corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)
# print(corpus)

Latent Dirichlet Allocation, LDA is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

In [ ]:
total_topics = 7
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics, passes=100)

In [ ]:
import sys

sys.stdout = open("ShowTopics_LDA", "w")

#Show first n important word in the topics:
lda.show_topics(total_topics,5)

In [ ]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [ ]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

g=sns.clustermap(df_lda.corr(), center=0, cmap="RdBu", metric='cosine', linewidths=.75, figsize=(12, 12))
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()
#plt.setp(ax_heatmap.get_yticklabels(), rotation=0)  # For y axis